In [2]:
!pip install streamlit
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report

# Streamlit UI
st.title('🧠 ANN Model Training & Evaluation Dashboard')
st.markdown("### 📌 Upload a dataset and configure the ANN model")

# Upload dataset
uploaded_file = st.file_uploader("📂 Upload CSV File", type=['csv'])

if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("### 🔍 Dataset Overview:", df.head())

    target_column = st.selectbox("🎯 Select Target Column", df.columns)
    feature_columns = [col for col in df.columns if col != target_column]

    # Data Preprocessing
    X = df[feature_columns]
    y = df[target_column]

    # Encode categorical variables
    for col in X.select_dtypes(include=['object']).columns:
        X[col] = LabelEncoder().fit_transform(X[col])

    if y.dtype == 'object':
        y = LabelEncoder().fit_transform(y)

    # Scaling numerical features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    # Splitting dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Sidebar for Model Configuration
    st.sidebar.header("⚙️ Model Configuration")
    num_layers = st.sidebar.slider("Hidden Layers", 1, 5, 3)
    neurons_per_layer = [st.sidebar.slider(f"Neurons in Layer {i+1}", 5, 100, 20) for i in range(num_layers)]
    activation_functions = [st.sidebar.selectbox(f"Activation for Layer {i+1}", ['relu', 'tanh', 'sigmoid']) for i in range(num_layers)]
    optimizer = st.sidebar.selectbox("Optimizer", ['adam', 'sgd', 'rmsprop'])
    loss_function = st.sidebar.selectbox("Loss Function", ['binary_crossentropy', 'categorical_crossentropy', 'mse'])
    dropout_rate = st.sidebar.slider("Dropout Rate", 0.0, 0.5, 0.2)
    epochs = st.sidebar.slider("Epochs", 10, 200, 50)
    batch_size = st.sidebar.slider("Batch Size", 8, 128, 32)
    output_activation = 'softmax' if len(set(y)) > 2 else 'sigmoid'

    # Train Model Button
    if st.button("🚀 Train Model"):
        model = Sequential()
        model.add(InputLayer(input_shape=(X_train.shape[1],)))

        for i in range(num_layers):
            model.add(Dense(neurons_per_layer[i], activation=activation_functions[i]))
            model.add(Dropout(dropout_rate))

        model.add(Dense(units=len(set(y)), activation=output_activation))

        model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

        with st.spinner('Training in progress...'):
            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)

        st.success("🎉 Training Complete!")

        # Loss & Accuracy Plots
        fig, ax = plt.subplots()
        ax.plot(history.history['loss'], label='Training Loss')
        ax.plot(history.history['val_loss'], label='Validation Loss')
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Loss")
        ax.legend()
        st.pyplot(fig)

        fig, ax = plt.subplots()
        ax.plot(history.history['accuracy'], label='Training Accuracy')
        ax.plot(history.history['val_accuracy'], label='Validation Accuracy')
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Accuracy")
        ax.legend()
        st.pyplot(fig)

        # Model Evaluation
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1) if output_activation == 'softmax' else (y_pred > 0.5).astype(int)
        report = classification_report(y_test, y_pred_classes, output_dict=True)
        st.write("### 📈 Classification Report:")
        st.json(report)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


2025-03-31 17:26:27.047 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.409 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-31 17:26:27.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.416 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 17:26:27.422 Thread 'MainThread': mi